In [1]:
%pip install docling quackling llama-index llama-index-llms-openllm
from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import PipelineOptions
import logging
import os
from dotenv import load_dotenv

_log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

Note: you may need to restart the kernel to use updated packages.


In [2]:
source = "/home/noelo/dev/instruct-injest/data/CELEX_32021R1173_EN_TXT.pdf"
converter = DocumentConverter(pipeline_options=PipelineOptions())
result = converter.convert_single(source)
print(len(result.pages))

/home/noelo/.pyenv/versions/3.10.14/envs/instvenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 42184.09it/s]
/home/noelo/.pyenv/versions/3.10.14/envs/instvenv/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to

49


In [3]:
load_dotenv()
print(os.getenv("API_KEY"))
print(os.getenv("LLM_URL"))

7d8b4497f2fe01e4c32a69902a914bbb
https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1


In [ ]:
# from quackling.llama_index.node_parsers import HierarchicalJSONNodeParser
# from quackling.llama_index.readers import DoclingPDFReader

# reader = DoclingPDFReader(parse_type=DoclingPDFReader.ParseType.JSON)
# node_parser = HierarchicalJSONNodeParser()

In [ ]:
# docs = reader.load_data(file_path=source)

In [ ]:
# from rich.pretty import pprint
# pprint(docs, max_length=2, max_string=250, max_depth=4)

In [ ]:
# from llama_index.core.ingestion import IngestionPipeline

# pipeline = IngestionPipeline(
#     transformations=[node_parser],
# )
# nodes = pipeline.run(documents=docs)
# print(len(nodes))
# # for x in nodes:
# #     print(x.text)

In [ ]:
# raw_text =''
# for x in nodes:
#     raw_text += x.text

In [4]:
raw_text = result.output.export_to_markdown()

In [ ]:
# from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained('nomic-ai/nomic-embed-text-v1.5')
# encoded_output = tokenizer(raw_text[:10000])

In [ ]:
# print(len(encoded_output.input_ids))

In [5]:
from __future__ import annotations

from typing import List, Optional

from pydantic import BaseModel, Field, constr


class QuestionsAndAnswer(BaseModel):
    question: constr(min_length=1)
    answer: constr(min_length=1)


class SeedExample(BaseModel):
    context: constr(min_length=1)
    questions_and_answers: List[QuestionsAndAnswer] = Field(
        ..., min_items=3, set=True
    )


class QNAModel(BaseModel):
    created_by: Optional[constr(min_length=1)] = None
    domain: Optional[constr(min_length=1)] = Field(
        None, examples=['Chemistry', 'History', 'Pop culture']
    )
    seed_examples: Optional[List[SeedExample]] = Field(
        None, min_items=5, set=True
    )


In [6]:
gen_prompt=f"you are a helpful writing assistant. Given the following context generate 3 question and answer pairs. Ensure that the questions can be answered by the context given.  Group the questions into different semantic contexts. Also output a description of the contexts. You MUST only answer using the following json schema: {QNAModel.schema_json()}"

In [7]:

%pip -q install semantic-router semantic-chunkers

from semantic_router.encoders.fastembed import FastEmbedEncoder

encoder = FastEmbedEncoder()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepsearch-toolkit 1.0.0 requires urllib3<2.0.0,>=1.26.8, but you have urllib3 2.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 31115.01it/s]


In [8]:
from semantic_chunkers import StatisticalChunker

chunker = StatisticalChunker(encoder=encoder,enable_statistics=True,plot_chunks=False,min_split_tokens=300, max_split_tokens=2000)

chunks = chunker(docs=[result.output.export_to_markdown()])


2024-09-08 13:03:11 INFO semantic_chunkers.utils.logger Single document exceeds the maximum token limit of 2000. Splitting to sentences before semantically merging.
  5%|▌         | 1/19 [00:00<00:15,  1.17it/s]

Chunking Statistics:
  - Total Documents: 64
  - Total Chunks: 6
  - Chunks by Threshold: 5
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 210
  - Maximum Token Size of Chunk: 653
  - Similarity Chunk Ratio: 0.83


 11%|█         | 2/19 [00:01<00:16,  1.03it/s]

Chunking Statistics:
  - Total Documents: 69
  - Total Chunks: 7
  - Chunks by Threshold: 6
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 120
  - Maximum Token Size of Chunk: 686
  - Similarity Chunk Ratio: 0.86


 16%|█▌        | 3/19 [00:02<00:15,  1.04it/s]

...

Chunking Statistics:
  - Total Documents: 71
  - Total Chunks: 4
  - Chunks by Threshold: 3
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 161
  - Maximum Token Size of Chunk: 453
  - Similarity Chunk Ratio: 0.75


 95%|█████████▍| 18/19 [00:20<00:01,  1.12s/it]

Chunking Statistics:
  - Total Documents: 69
  - Total Chunks: 4
  - Chunks by Threshold: 3
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 238
  - Maximum Token Size of Chunk: 664
  - Similarity Chunk Ratio: 0.75


100%|██████████| 19/19 [00:20<00:00,  1.08s/it]

Chunking Statistics:
  - Total Documents: 74
  - Total Chunks: 3
  - Chunks by Threshold: 2
  - Chunks by Max Chunk Size: 0
  - Last Chunk: 1
  - Minimum Token Size of Chunk: 203
  - Maximum Token Size of Chunk: 583
  - Similarity Chunk Ratio: 0.67


In [24]:
# chunker.print(chunks[2])
i=0
for ch in chunks[0]:
    i+=1
    print(len(ch.content))

print(i)

2440
1112
2249
3824
3669
3369
2001
3885
2329
3057
3254
3047
5146
1735
2831
1942
5561
2665
3271
3460
5161
2502
3668
3783
1931
2207
1945
2041
2774
1490
2076
1646
1897
2306
1734
2311
1824
1607
1823
1719
1984
2294
2217
2889
2183
1982
3029
2225
1990
1757
3770
3062
1952
1911
2810
3289
1748
1853
2342
1908
4251
2405
1816
2301
1954
3418
2493
3318
2322
1107
70


In [34]:
from llama_index.llms.openllm import OpenLLM

gen_prompt2=f"you are a helpful writing assistant. Given the following context generate 3 question and answer pairs. Ensure that the questions can be answered by the context given.  Also output a description of the contexts. All output MUST only use the following json schema: {QNAModel.schema_json()} \n Context: "

In [35]:
llm = OpenLLM(
    model="mistral-7b-instruct", api_base=os.getenv("LLM_URL"),
    api_key=os.getenv("API_KEY")
)

comp_resp=''

for ch in chunks[0]:

    for it in llm.stream_complete(gen_prompt2+ch.content,max_tokens=4000,timeout=120.0):
        comp_resp+=it.text
    print(it)

INFO:httpx:HTTP Request: POST https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com/v1/completions "HTTP/1.1 200 OK"


 interest which are not covered by other Union programmes. (5) The European High Performance Computing Joint Undertaking ('the Joint Undertaking') is a partnership between the European Union, Member States and the private sector, established to ensure the development of a world-class supercomputing infrastructure in Europe. The Joint Undertaking will be implemented through Horizon Europe.

 SeedExample:
 {
 "context": "## REGULATIONS ## COUNCIL REGULATION (EU) 2021/1173 of 13 July 2021 on establishing the European High Performance Computing Joint Undertaking and repealing Regulation (EU) 2018/1488",
 "questions_and_answers": [
 {
 "question": "What is the name of the partnership established to ensure the development of a world-class supercomputing infrastructure in Europe?",
 "answer": "The European High Performance Computing Joint Undertaking"
 },
 {
 "question": "What is the purpose of the Digital Europe Programme according to the context?",
 "answer": "The Digital Europe Programme s

INFO:httpx:HTTP Request: POST https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com/v1/completions "HTTP/1.1 200 OK"




 SeedExample:
{
  "context": "EU Regulations related to infrastructure development in the transport, telecommunications, and energy sectors",
  "questions_and_answers": [
    {
      "question": "What is the name of the regulation that established Horizon Europe?",
      "answer": "Regulation (EU) 2021/695 of the European Parliament and of the Council of 28 April 2021"
    },
    {
      "question": "Which regulation is responsible for the development and construction of new infrastructures and services in the transport, telecommunications, and energy sectors?",
      "answer": "The regulations mentioned in the context"
    },
    {
      "question": "What is the name of the regulation that established the Digital Europe Programme?",
      "answer": "Regulation (EU) 2021/694 of the European Parliament and of the Council of 29 April 2021"
    }
  ]
}

 QuestionsAndAnswer:
[
  {
    "question": "What is the name of the regulation that established Horizon Europe?",
    "answer": "Regula

INFO:httpx:HTTP Request: POST https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com/v1/completions "HTTP/1.1 200 OK"


...

 The Union's share of access time to each EuroHPC supercomputer shall be allocated to users in accordance with the rules set out in Article 18.

 SeedExample:
{
  "context": "3. Users residing, established or located in a Member State or in a third country associated to Horizon 2020 shall be granted access to the Union's share of access time of the supercomputers acquired by the EuroHPC Joint Undertaking established by Regulation (EU) 2018/1488. 4. Users residing, established or located in a Member State or in a third country associated to the Digital Europe Programme or to Horizon Europe shall be granted the Union's share of access time to EuroHPC supercomputers acquired after 2020. 5. In duly justified cases, taking into account the interests of the Union, the Governing Board shall decide to grant access time to EuroHPC supercomputers to entities residing, established or located in any third country and to international organisations.",
  "questions_and_answers": [
    {
      "quest

INFO:httpx:HTTP Request: POST https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com/v1/completions "HTTP/1.1 200 OK"




 SeedExample:
{
  "context": "Each Participating State where a hosting entity is established or each Participating State in a hosting consortium shall be allocated the remaining access time to each EuroHPC supercomputer. In the case of a hosting consortium, the Participating States shall agree among themselves the distribution of access time to the supercomputer.",
  "questions_and_answers": [
    {
      "question": "Who is allocated the remaining access time to each EuroHPC supercomputer?",
      "answer": "Each Participating State where a hosting entity is established or each Participating State in a hosting consortium"
    },
    {
      "question": "What happens in the case of a hosting consortium regarding the distribution of access time to the supercomputer?",
      "answer": "The Participating States shall agree among themselves the distribution of access time to the supercomputer."
    },
    {
      "question": "What is the role of the Governing Board in the distribution of

INFO:httpx:HTTP Request: POST https://mistral-7b-instruct-v0-3-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com/v1/completions "HTTP/1.1 200 OK"


ReadTimeout: The read operation timed out